In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, LSTM, GRU, Embedding, Concatenate, Conv1D, GlobalMaxPooling1D, MaxPooling1D, GlobalAveragePooling1D, BatchNormalization, TimeDistributed
import tensorflow.keras.backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.models import load_model
import tensorflow.keras

import math
from lifelines import KaplanMeierFitter
from lifelines import CoxPHFitter
from lifelines.utils import concordance_index
from sklearn.preprocessing import StandardScaler
from scipy import stats

from preprocessor.nnet_survival import nnet_survival

%load_ext autoreload
%autoreload 2

2023-06-25 17:50:20.515230: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-25 17:50:21.863668: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/biotools/python/3.8.1/lib:/usr/local/biotools/cuda/11.7/lib64:/usr/local/biotools/cuda/11.7/nccl/lib:
2023-06-25 17:50:21.863756: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/biotoo

In [2]:

# Determine CSV, label, and key columns
CSV_COLUMNS = [
    "patientid",
    "task_1",
    "task_2",
    "centerid",
    "gender",
    "age",
    "weight",
    "tobacco",
    "alcohol",
    "performance_status",
    "hpv_status",
    "surgery",
    "chemotherapy",
    "relapse",
    "rfs",    
]
LABEL_COLUMN = "weight_pounds"

NUMERICAL_COLUMNS = ["age", "weight"]
CATEGORICAL_COLUMNS = ["centerid", "gender", "tobacco", "alcohol",
                       "performance_status", "hpv_status",
                       "surgery", "chemotherapy", ]
UNWANTED_COLS = ["patientid", "task_1", "task_2"]

# Set default values for each CSV column.
# Treat is_male and plurality as strings.
DEFAULTS = [[0.0], ["null"], [0.0], ["null"], [0.0]]


def features_and_labels(row_data):
    """Splits features and labels from feature dictionary.

    Args:
        row_data: Dictionary of CSV column names and tensor values.
    Returns:
        Dictionary of feature tensors and label tensor.
    """
    label = row_data.pop(LABEL_COLUMN)

    return row_data, label  # features, label


def load_dataset(pattern, batch_size=1, mode=tf.estimator.ModeKeys.EVAL):
    """Loads dataset using the tf.data API from CSV files.

    Args:
        pattern: str, file pattern to glob into list of files.
        batch_size: int, the number of examples per batch.
        mode: tf.estimator.ModeKeys to determine if training or evaluating.
    Returns:
        `Dataset` object.
    """
    # Make a CSV dataset
    dataset = tf.data.experimental.make_csv_dataset(
        file_pattern=pattern,
        batch_size=batch_size,
        column_names=CSV_COLUMNS,
        column_defaults=DEFAULTS,
    )

    # Map dataset to features and label
    dataset = dataset.map(map_func=features_and_labels)  # features, label

    # Shuffle and repeat for training
    if mode == tf.estimator.ModeKeys.TRAIN:
        dataset = dataset.shuffle(buffer_size=1000).repeat()

    # Take advantage of multi-threading; 1=AUTOTUNE
    dataset = dataset.prefetch(buffer_size=1)

    return dataset


def create_input_layers():
    """Creates dictionary of input layers for each feature.

    Returns:
        Dictionary of `tf.Keras.layers.Input` layers for each feature.
    """
    deep_inputs = {
        colname: tf.keras.layers.Input(
            name=colname, shape=(1,), dtype="float32"
        )
        for colname in NUMERICAL_COLUMNS
    }

    wide_inputs = {
        colname: tf.keras.layers.Input(name=colname, shape=(1,), dtype="string")
        for colname in CATEGORICAL_COLUMNS
    }

    inputs = {**wide_inputs, **deep_inputs}

    return inputs


def transform(inputs, nembeds):
    """Creates dictionary of transformed inputs.

    Returns:
        Dictionary of transformed Tensors
    """

    deep = {}
    wide = {}

    buckets = {
        "mother_age": np.arange(15, 45, 1).tolist(),
        "gestation_weeks": np.arange(17, 47, 1).tolist(),
    }
    bucketized = {}

    for numerical_column in NUMERICAL_COLUMNS:
        deep[numerical_column] = inputs[numerical_column]
        bucketized[numerical_column] = tf.keras.layers.Discretization(buckets[numerical_column])(inputs[numerical_column])
        wide[f"btk_{numerical_column}"] = tf.keras.layers.CategoryEncoding(
            num_tokens=len(buckets[numerical_column]) + 1, output_mode="one_hot"
        )(bucketized[numerical_column])

    crossed = tf.keras.layers.experimental.preprocessing.HashedCrossing(
        num_bins=len(buckets["mother_age"]) * len(buckets["gestation_weeks"])
    )((bucketized["mother_age"], bucketized["gestation_weeks"]))

    deep["age_gestation_embeds"] = tf.keras.layers.Flatten()(
        tf.keras.layers.Embedding(
            input_dim=len(buckets["mother_age"])
            * len(buckets["gestation_weeks"]),
            output_dim=nembeds,
        )(crossed)
    )

    vocab = {
        "is_male": ["True", "False", "Unknown"],
        "plurality": [
            "Single(1)",
            "Twins(2)",
            "Triplets(3)",
            "Quadruplets(4)",
            "Quintuplets(5)",
            "Multiple(2+)",
        ],
    }

    for categorical_column in CATEGORICAL_COLUMNS:
        wide[categorical_column] = tf.keras.layers.StringLookup(
            vocabulary=vocab[categorical_column], output_mode="one_hot"
        )(inputs[categorical_column])

    return wide, deep

def get_model_outputs(wide_inputs, deep_inputs, dnn_hidden_units):
    """Creates model architecture and returns outputs.

    Args:
        wide_inputs: Dense tensor used as inputs to wide side of model.
        deep_inputs: Dense tensor used as inputs to deep side of model.
        dnn_hidden_units: List of integers where length is number of hidden
            layers and ith element is the number of neurons at ith layer.
    Returns:
        Dense tensor output from the model.
    """
    # Hidden layers for the deep side
    layers = [int(x) for x in dnn_hidden_units.split()]
    deep = deep_inputs
    for layerno, numnodes in enumerate(layers):
        deep = tf.keras.layers.Dense(
            units=numnodes, activation="relu", name=f"dnn_{layerno + 1}"
        )(deep)
    deep_out = deep

    # Linear model for the wide side
    wide_out = tf.keras.layers.Dense(
        units=10, activation="relu", name="linear"
    )(wide_inputs)

    # Concatenate the two sides
    both = tf.keras.layers.Concatenate(name="both")([deep_out, wide_out])

    # Final output is a linear activation because this is regression
    output = tf.keras.layers.Dense(units=1, activation="linear", name="weight")(
        both
    )

    return output


def rmse(y_true, y_pred):
    """Calculates RMSE evaluation metric.

    Args:
        y_true: tensor, true labels.
        y_pred: tensor, predicted labels.
    Returns:
        Tensor with value of RMSE between true and predicted labels.
    """
    return tf.sqrt(tf.reduce_mean(tf.square(y_pred - y_true)))


def build_wide_deep_model(dnn_hidden_units=[64, 32], nembeds=3):
    """Builds wide and deep model using Keras Functional API.

    Returns:
        `tf.keras.models.Model` object.
    """
    # Create input layers
    inputs = create_input_layers()

    # transform raw features for both wide and deep
    wide, deep = transform(inputs, nembeds)

    # The Functional API in Keras requires: LayerConstructor()(inputs)
    wide_inputs = tf.keras.layers.Concatenate()(wide.values())
    deep_inputs = tf.keras.layers.Concatenate()(deep.values())

    # Get output of model given inputs
    output = get_model_outputs(wide_inputs, deep_inputs, dnn_hidden_units)

    # Build model and compile it all together
    model = tf.keras.models.Model(inputs=inputs, outputs=output)
    model.compile(optimizer="adam", loss="mse", metrics=[rmse, "mse"])

    return model


# Instantiate the HyperTune reporting object
hpt = hypertune.HyperTune()

# Reporting callback
class HPTCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        global hpt
        hpt.report_hyperparameter_tuning_metric(
            hyperparameter_metric_tag='val_rmse',
            metric_value=logs['val_rmse'],
            global_step=epoch)

        
def train_and_evaluate(args):
    model = build_wide_deep_model(args["nnsize"], args["nembeds"])
    print("Here is our Wide-and-Deep architecture so far:\n")
    print(model.summary())

    trainds = load_dataset(
        args["train_data_path"],
        args["batch_size"],
        tf.estimator.ModeKeys.TRAIN)

    evalds = load_dataset(
        args["eval_data_path"], 1000, tf.estimator.ModeKeys.EVAL)
    if args["eval_steps"]:
        evalds = evalds.take(count=args["eval_steps"])

    num_batches = args["batch_size"] * args["num_epochs"]
    steps_per_epoch = args["train_examples"] // num_batches

    checkpoint_path = os.path.join(args["output_dir"], "checkpoints/babyweight")
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_path, verbose=1, save_weights_only=True)

    history = model.fit(
        trainds,
        validation_data=evalds,
        epochs=args["num_epochs"],
        steps_per_epoch=steps_per_epoch,
        verbose=2,  # 0=silent, 1=progress bar, 2=one line per epoch
        callbacks=[cp_callback, HPTCallback()])

    EXPORT_PATH = os.path.join(
        args["output_dir"], datetime.datetime.now().strftime("%Y%m%d%H%M%S"))
    tf.saved_model.save(
        obj=model, export_dir=EXPORT_PATH)  # with default serving function
    
    print("Exported trained model to {}".format(EXPORT_PATH))

NameError: name 'tf' is not defined

In [ ]:

###################################################################################
#Flexible model: Convolutional neural network using MNIST data
#Uses some code from https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py
#For this example, we use images of the handwritten numbers 0 through 4.
#Larger numbers have shorter average survival. Task: Given an image of a number, predict survival curve.
#Described in paper.


# We need to split the data set into train and test, and fill in time & event for both groups
train_data = pd.read_csv("../data/task2/train_data.csv")
test_data = pd.read_csv("../data/task2/test_data.csv")
time = train_data['rfs']
event = train_data['relapse']
timeTest = test_data['rfs']
eventTest = test_data['relapse']
x_train = train_data[['centerid',
                      'age',
                      'weight',
                      'tobacco',
                      'alcohol',
                      'performance_status',
                      'hpv_status',
                      'surgery',
                      'chemotherapy',
                      'gender_m']].values
x_test = test_data[['centerid',
                      'age',
                      'weight',
                      'tobacco',
                      'alcohol',
                      'performance_status',
                      'hpv_status',
                      'surgery',
                      'chemotherapy',
                      'gender_m']].values

#Convert event data to array format

halflife=365.*4
breaks=-np.log(1-np.arange(0.0,0.96,0.05))*halflife/np.log(2) 
#breaks=np.concatenate((np.arange(0,200,10),np.arange(200,1001,25)))

n_intervals=len(breaks)-1
timegap = breaks[1:] - breaks[:-1]
y_train_array=nnet_survival.make_surv_array(time,event,breaks)

#Train model
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(1)

model = Sequential()
model.add(Dense(64, input_shape=(10,),activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.25))

prop_hazards=0
if prop_hazards:
	model.add(Dense(1, use_bias=0, kernel_initializer='zeros'))
	model.add(nnet_survival.PropHazards(n_intervals))
else:
	model.add(Dense(n_intervals, kernel_initializer='zeros', bias_initializer='zeros'))
	model.add(Activation('sigmoid'))

model.compile(loss=nnet_survival.surv_likelihood(n_intervals), optimizer=tf.keras.optimizers.Adam())
early_stopping = EarlyStopping(monitor='loss', patience=50)
history=model.fit(x_train, y_train_array, batch_size=64, epochs=10000, verbose=1, callbacks=[early_stopping])

#Training set results
y_pred=model.predict(x_train,verbose=0)


#discrimination (C-index)
oneyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
print(concordance_index(time,oneyr_surv,event))

#calibration plot
days_plot = 365*4
plt.subplot(1,1,1)
#plt.subplot(1, 2, 1)
kmf = KaplanMeierFitter()
matplotlib.style.use('default')
actual = []
predicted = []
kmf.fit(time, event_observed=event)
actual.append(plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='--',c='r'))
pred_surv=np.mean(np.cumprod(y_pred, axis=1),axis=0)
predicted.append(plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='-',c='b'))
#print(i, kmf.median_)

plt.xticks(np.arange(0, days_plot+0.0001, 200))
plt.yticks(np.arange(0, 1.0001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Proportion surviving')
plt.title('Training set calibration')
#plt.show()

#Test set results
y_pred=model.predict(x_test,verbose=0)

#discrimination (C-index)
oneyr_surv=np.cumprod(y_pred[:,0:np.nonzero(breaks>365)[0][0]], axis=1)[:,-1]
print(concordance_index(timeTest,oneyr_surv,eventTest))

#discrimination of perfect model that uses actual digit as survival time predictor
#print(concordance_index(timeTest,-y_test.astype('float'),eventTest))

In [ ]:


#calibration
""" plt.subplot(1, 2, 2)
kmf = KaplanMeierFitter()
matplotlib.style.use('default')
actual = []
predicted = []
for i in range(num_classes):
	kmf.fit(timeTest[y_test==i], event_observed=eventTest[y_test==i])
	actual.append(plt.plot(kmf.survival_function_.index.values, kmf.survival_function_.KM_estimate,ls='--',c='C'+str(i)))
	pred_surv=np.mean(np.cumprod(y_pred[y_test==i,:], axis=1),axis=0)
	predicted.append(plt.plot(breaks,np.concatenate(([1],pred_surv)),ls='-',c='C'+str(i)))

plt.xticks(np.arange(0, days_plot+0.0001, 200))
plt.yticks(np.arange(0, 1.0001, 0.125))
plt.xlim([0,days_plot])
plt.ylim([0,1])
plt.xlabel('Follow-up time (days)')
plt.ylabel('Proportion surviving')
plt.legend(['0: Actual','0: Predicted',
	'1: Actual','1: Predicted',
	'2: Actual','2: Predicted',
	'3: Actual','3: Predicted',
	'4: Actual','4: Predicted'])
plt.title('Test set calibration')
plt.show() """
